In [1]:
import datetime

import pandas as pd
from feast import FeatureStore

In [2]:
data = pd.read_csv("./data/features.csv", header="infer", sep=";")

In [3]:
data

,token,create_timestamp,event_timestamp,prev_1days,prev_2days,prev_3days,prev_4days,prev_5days,prev_6days,prev_7days,prev_8days,prev_9days,prev_10days,prev_11days,prev_12days,prev_13days
0,btc,2021-06-11 11:00:00+00:00,2021-06-26 11:08:55.222,1,1,0,0,0,0,1,1,0,0,0,1,0


In [4]:
data['create_timestamp'] = data['create_timestamp'].apply(datetime.datetime.fromisoformat)
data['event_timestamp'] = data['event_timestamp'].apply(datetime.datetime.fromisoformat)

In [5]:
# use this data in "feast apply"
data.to_parquet("./data/features.parquet")

In [6]:
# entity_df generally comes from upstream systems
entity_data = pd.DataFrame.from_dict({
    "token": ["btc","btc","btc","btc"],
    "event_timestamp": [
        datetime.datetime(2021, 4, 12, 10, 59, 42),
        datetime.datetime(2021, 4, 12, 8,  12, 10),
        datetime.datetime(2021, 4, 12, 16, 40, 26),
        datetime.datetime(2021, 4, 12, 15, 1 , 12),
    ]
})

In [7]:
entity_data.head()

,token,event_timestamp
0,btc,2021-04-12 10:59:42
1,btc,2021-04-12 08:12:10
2,btc,2021-04-12 16:40:26
3,btc,2021-04-12 15:01:12


In [8]:
store = FeatureStore(repo_path=".")
training_data = store.get_historical_features(
    entity_df=entity_data, 
    features = [
        "token_hourly_features:prev_3days",
        "token_hourly_features:prev_4days",
        "token_hourly_features:prev_5days",
    ],
)

In [9]:
training_data.to_df()

,token,event_timestamp,prev_3days,prev_4days,prev_5days
